In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("titanic").getOrCreate()

In [3]:
df = spark.read.csv("titanic.csv",inferSchema=True,
                   header=True)

In [5]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [6]:
df.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [7]:
my_cols=df.select(['Survived','Pclass','Sex',
 'Age',
 'SibSp',
 'Parch','Fare','Embarked'])

In [8]:
my_final_data = my_cols.na.drop()

In [9]:
 from pyspark.ml.feature import (VectorAssembler,VectorIndexer,OneHotEncoder,StringIndexer)

In [23]:
gender_indexer = StringIndexer(inputCol="Embarked",outputCol="EmbarkIndex")
gender_encoder = OneHotEncoder(inputCol="EmbarkIndex",outputCol="EmbarkVec")

In [24]:
embark_indexer = StringIndexer(inputCol="Sex",outputCol="SexIndex")
embark_encoder = OneHotEncoder(inputCol="SexIndex",outputCol="SexVec")

In [25]:
assembler = VectorAssembler(inputCols=["Pclass","SexVec","EmbarkVec","Age","SibSp","Parch","Fare"],outputCol="features")

In [26]:
from pyspark.ml.classification import LogisticRegression

In [27]:
from pyspark.ml import Pipeline

In [28]:
log_reg_titanic_ = LogisticRegression(featuresCol="features",labelCol="Survived")

In [29]:
pipeline = Pipeline(stages=[gender_indexer,embark_indexer,
                            gender_encoder,embark_encoder,
                            assembler,log_reg_titanic_])

In [31]:
train_data, test_data = my_final_data.randomSplit([0.7,0.3])

In [32]:
fit_model = pipeline.fit(train_data)

In [33]:
results=fit_model.transform(test_data)

In [34]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [36]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol="prediction",labelCol="Survived")

In [37]:
results.select("Survived","prediction").show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 20 rows



In [38]:
AUC = my_eval.evaluate(results)

In [39]:
AUC

0.7920054200542005